# CSCI 3155: Assignment 6 (45 points)

Topics Covered: Function calls in Lettuce

__Name__: Ben Jacobs

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (45 Points): Multiple Argument Functions

In class, we have explored just single argument functions in lettuce. In this problem, we will 
explore, multiple argument functions.

Consider: 

~~~
let foo1 = function (x, y)
            x - 2 * y
          in 
       foo1(10, 15)
~~~

this code should return -20

We will allow zero arguments as well.

~~~
let x = 5 in 
let bar1 = function()
           x 
           in 
    bar1()
~~~

This code should return 5.

## 1 A (7 points): Extending the Abstract Syntax

Consider the grammar specification we have seen thus far.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & FunDef( \mathbf{Identifier}^*, \mathbf{Expr}) & \text{Note multiple parameters possible} \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call - expr(expr1, ... , exprn)} \\
 & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
\end{array}$$

Write the scala definition for `FunDef` and `FunCall`. Please use lists to implement the Kleene Star.

In [13]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(s: List[String], e1: Expr) extends Expr
case class FunCall(e1: Expr, e2: List[Expr]) extends Expr
case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Let
defined class FunDef
defined class FunCall
defined class TopLevel


In [14]:
//BEGIN TEST
// TEST Two arguments
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
val fun1 = FunDef(List("x", "y"), Plus(x, y))
val l1 = Let("foo", fun1, FunCall(foo, List(Const(10.0), Const(20.0))))
val p1 = TopLevel(l1)
passed(3)
//END TEST


*** Tests Passed (3 points) ***


null

In [15]:
//BEGIN TEST
//Test zero arguments
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List())
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p2 = TopLevel(l3)
passed(4)
//END TEST


*** Tests Passed (4 points) ***


null

## 1B (8 points): Define Closure for Function Calls with Zero or More Args.

We will now redefine closures for functions with zero or more args. When our function had one argument, our closures were defined as `Closure(id, expr, env)`. We would now like to define closures as
`Closure( [id1, ..., idn], expr, env)` where 
  - `id1..., idn` are the list of arguments for the function to be called. 
  - `expr` is the body of the function and 
  - `env` is the stored environment for static scoping.
 
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & num(\mathbf{Double}) \\
 & \Rightarrow & closure(\mathbf{String}^*, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & error \\
 \end{array}$$
 
 For __Environment__ please use a scala immutable map from __String__ to __Value__

In [ ]:
sealed trait Value
case class Num(d: Double) extends Value
case object Error extends Value
case class Closure(s: List[String], e1: Expr, env: Map[String, Value]) extends Value

In [ ]:
//BEGIN TEST
val v1 = Closure(List("x", "y", "z"), Plus(x, y), Map.empty)
val v2 = Closure(Nil, Const(15), Map.empty)
val env1: Map[String, Value] = Map("x" -> Num(25.0), "y"-> v2)
val v3 = Closure(List("x"), Plus(x, Const(25)), env1 )
passed(8)
//END TEST

## 1 C (30 Points) Build an Interpreter

Build an interpreter using the following semantic rules. Ensure that your interpreter correctly deals with the cases that give rise to error by throwing an exception.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody}, \color{red}{\sigma_{cl} \circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
           
Let us interpret this rule. Read the points below carefully.
   - __Purpose__ : Evaluate an expression of the form `FunCall(e, [e1,...,ek])` where `e` is the expr for the called function, and `e1, ..., ek` are exprs for the arguments of this call. There are $k$ arguments.
     - (A) Evaluating `e` must yield a closure of the form $\text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}})$.
     - (B) The number of arguments for the closure $n$ must equal that of the function call $k$.
     - (C) Evaluating each of the $k$ arguments `ei` for $i = 1, \ldots, k$ must yield $v_i$ where $v_i$ is not error.
     - (D) Then the result of evaluating the call is the same as that of evaluating `fBody` under the environment $\color{red}{\sigma_{cl}}$ extended by mapping the formal parameters `id1.., idk` to $v_1, \ldots, v_k$, respectively.


We can write some error rules.

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Closure}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
           
  
$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$
           
           
**Important:** The use of `var` and `while/for` loop is not allowed. You can use the functions map, foldLeft and zip over lists.

**Hint** The following function will be very useful for you. Try it out and see what it does!

~~~
val l1 = List(10, 15, 20, 25, 30)
val l2 = List("a", "b", "c", "d", "e")
val l3 = l1.zip(l2)
println(l3)
~~~
 

In [5]:
class ErrorException(s:String) extends Exception(s){}  


def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Map[String, Value]): Value = {
    def addValues(v1: Value, v2: Value): Value = 
        Num ( valueToNumber(v1) + valueToNumber(v2) )
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => {
            ??? // YOUR CODE HERE
        }
        case Plus(e1,e2) => {
            val v1 = eval(e1, env)
            val v2 = eval(e2, env)
            addValues(v1,v2)
        }
        case Let(id, e1, e2) => {
            ??? // YOUR CODE HERE
        }
        
        case FunDef(idList, e) => {
            Closure(idList, e, env)
        }
        
        case FunCall(e, eList) => {
            ??? // YOUR CODE HERE
        }
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, Map.empty)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}


<console>: 19

In [6]:
//BEGIN TEST
// TEST Two arguments
val x = Ident("x")
val y = Ident("y")
val foo = Ident("foo")
// function(x, y)  x + y
val fun1 = FunDef(List("x", "y"), Plus(x, y))
//let foo = function(x,y) x+ y in foo(10, 20)
val l1 = Let("foo", fun1, FunCall(foo, List(Const(10.0), Const(20.0))))
//Program
val p1 = TopLevel(l1)
val v1 = evalProgram(p1)
println(s"Your program evaluated to $v1")
assert(v1 == Num(30.0), "Test 1 Failed")
passed(5)
//END TEST

<console>: 92

In [7]:
//BEGIN TEST
// TEST three arguments
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo2")
// function(x, y,z)  x + y+z+z
val fun2 = FunDef(List("x", "y", "z"), Plus(Plus(Plus(x, y), z), z))
//let foo = function(x,y) x+ y in foo(10, 20, 30)
val l2 = Let("foo2", fun2, FunCall(foo, List(Const(10.0), Const(20.0), Const(30.0))))
//Program
val p2 = TopLevel(l2)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(90.0), "Test 2 Failed")
passed(5)
//END TEST

<console>: 92

In [8]:
//BEGIN TEST
//Test zero arguments
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List())
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p2 = TopLevel(l3)
val v3 = evalProgram(p2)
println(s"Your program evaluated to $v3")
assert(v3 == Num(10.0), "Test 3 Failed")
passed(5)
//END TEST

<console>: 92

In [9]:
//BEGIN TEST
//Evaluate with wrong number of args
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List(Const(1.0)))
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p4 = TopLevel(l3)
val v4 = evalProgram(p4)
assert(v4 == Error, "Test 4 failed -- your program should have detected that arguments were mismatched")
passed(5)
//END TEST

<console>: 92

In [10]:
//BEGIN TEST
// TEST scoping
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo")

// function(x, y)  x + y + x + z 
val fun = FunDef(List("x", "y"), Plus(Plus(Plus(x, y), x),z))
val funcall = FunCall(foo, List(Const(10.0), Const(30.0)))
// let x = 1 in
//     let z = 50 in
//       let foo = fun(x,y) x + y + x + z in
//         let = 2 in
//           f(10,30)
val l4 = Let("x", Const(1.0), Let("z", Const(50.0), Let("foo", fun, Let("y", Const(2.0), funcall))))
//Program
val p2 = TopLevel(l4)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(100.0), "Test 5 Failed == you did not evaluate static scoping correctly")
passed(5)
//END TEST

<console>: 92

In [11]:
//BEGIN TEST
//Evaluate with non function call
val x = Ident("x")
val fcall = FunCall(x, List(Const(1.0), Const(5.0), Const(20.0)))
val l3 = Let("x", Const(10), fcall)
val p5 = TopLevel(l3)
val v5 = evalProgram(p5)
assert(v5 == Error, "Test 6 failed -- your eval should have detected that a function call is made on a expression that is not a function")
passed(5)
//END TEST

<console>: 92